In [1]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType,StringType
from pyspark.sql.functions import monotonicallyIncreasingId
import pulp
from pulp import *


def predicted(Predicted):
    if Predicted < 0.0005:
        Predicted = 0.0
    else:
        Predicted
    return Predicted


def EPB(EPB):
    if EPB < 0.0 :
        EPB = EPB*(-1)
    else:
        EPB
    return EPB

def objective_fun(EPB,Predicted):
    return EPB * Predicted

def lower_limit_fun(EPB,Average_Weekly_Traffic):
    if EPB >= 100.0:
        limit=0.5*Average_Weekly_Traffic
    elif EPB>=50.0 and EPB <100.0:
        limit=0.2*Average_Weekly_Traffic
    else:
        limit=0.2*Average_Weekly_Traffic
    return limit 


def var_name(id):
    return "T_" + str(id)

objective_fun_udf = udf(objective_fun, DoubleType())

lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())

var_name_udf = udf(var_name,StringType())

EPB_udf = udf(EPB, DoubleType())

Predicted_udf = udf(predicted , DoubleType())

Optimization_T_ALL = sqlContext.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true') \
.option('mode', 'DROPMALFORMED').load('/home/affine/Expedia/Data/Optimization_Sample_AD.csv')

Optimization_T_predict = Optimization_T_ALL.withColumn("Predicted_new",Predicted_udf(Optimization_T_ALL.Predicted))

Optimization_T_traffic =  Optimization_T_predict.filter(Optimization_T_predict.Traffic!=0)

Optimization_T = Optimization_T_traffic.withColumn("EPB_new",EPB_udf(Optimization_T_traffic.EPB))


Optimization_T_Temp = Optimization_T.withColumn("Objective_Fun",objective_fun_udf(Optimization_T.EPB_new ,Optimization_T.Predicted_new)).withColumn("Lower_Limit", lower_limit_fun_udf(Optimization_T.EPB_new , Optimization_T.Average_Weekly_Traffic)).withColumn("id", monotonicallyIncreasingId())
Optimization_T_Final = Optimization_T_Temp.withColumn("var_name" ,var_name_udf(Optimization_T_Temp.id))

Optimization_T_Final.show(1)






+--------+--------------------+-------------+--------------------+--------------------+----------+-----------+---------+---------------+-----------------+-----------+------------------+----------+---------+-----------+------------------+-------+----+----+----------------------+-------+------------+-----+---------+-------------+-------+-------------+-----------+---+--------+
|Hotel_ID|              Region|Region_Bucket|       Market_Bucket|              Market|Model_Name|City_Bucket|     City|     Hotel_Type|Hotel_Type_Bucket|Star_Rating|Star_Rating_Bucket|LOS_Bucket|BW_Bucket|seller_rank|Seller_Rank_Bucket|Traffic|Cost| CPC|Average_Weekly_Traffic|Booking|Gross_Profit|  EPB|Predicted|Predicted_new|EPB_new|Objective_Fun|Lower_Limit| id|var_name|
+--------+--------------------+-------------+--------------------+--------------------+----------+-----------+---------+---------------+-----------------+-----------+------------------+----------+---------+-----------+------------------+-------+-

In [2]:
def f(Canada_DF):
    
    Canada_DF = list(Canada_DF)
    canada_Count = len (Canada_DF)
    
    region_name = [Canada_DF[i][1][5] for i in range(canada_Count) if i == 0]
    
    t_list = [Canada_DF[i][1][29] for i in range(canada_Count)]
    
    objective_list = [Canada_DF[i][1][26] for i in range(canada_Count)]
    
    final_objective = {t_list[i]: objective_list[i] for i in range(0, canada_Count)}
    
    lower_limit_list = [Canada_DF[i][1][27] for i in range(canada_Count)]
    upper_limit_list = [Canada_DF[i][1][19] for i in range(canada_Count)]
    
    lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, canada_Count)}
    upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, canada_Count)}
    
    cnt =0.3
    lp_status = 'Start'
    
    while ((lp_status != "Optimal")  & (cnt <=2.0)):
    
        a = cnt

        last_row =  [(Canada_DF[i][1][18] - a*Canada_DF[i][1][26]) for i in range(canada_Count)]

        final_constraint = {t_list[i]: last_row[i] for i in range(0, canada_Count)}

        prob = LpProblem("The Optimization Problem", LpMaximize)

        lp_var = {}

        for variable in t_list:

            variable1 = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
            lp_var[str(variable)] = variable1

        prob += lpSum([final_objective[i]*lp_var[i] for i in t_list]) 
        prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0

        prob.solve()

        var_values = {"start":0}

        for v in prob.variables():
            var_values.update({v.name[2:]:v.varValue})

        lp_status = LpStatus[prob.status]
       
        results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}

        cnt = cnt+.01

    yield  results



In [3]:
def part_id(x):
    if x == "Canada":
        return 1.0
    elif x == "Mexico":
        return 2.0
    elif x == "CARIBBEAN":
        return 3.0
    elif x == "US":
        return 4.0
    elif x == "North America":
        return 5.0
    elif x == "Other Region":
        return 6.0
    
part_id_udf = udf(part_id,DoubleType())

Optimization_T_Final_temp = Optimization_T_Final.withColumn("part_id",part_id_udf(Optimization_T_Final.Model_Name))

pairs = Optimization_T_Final_temp.rdd.map(lambda x : (x[30],x))

pairs_part = pairs.partitionBy(6)

test = pairs_part.mapPartitions(f)

In [5]:
final_results = test.collect()

In [52]:
import pandas
def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_temp = pandas.DataFrame(cnt_list)
    df = sqlContext.createDataFrame(df)
    
    return df 

In [54]:
from pyspark.sql.types import *
field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
schema = StructType(field)

df1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

for i in range(len(final_results)):
    df1 = df1.unionAll(df_create(final_results[i]))
    
   

In [55]:
df1.show()

+----------+---------+----------+------------+
|    var_id|var_value|Efficiency|       Model|
+----------+---------+----------+------------+
|8589947279|      1.0|       0.7|Other Region|
|8589953158|      0.4|       0.7|Other Region|
|     51923|      1.0|       0.7|Other Region|
|8589965897|      0.2|       0.7|Other Region|
|8589934630|      0.2|       0.7|Other Region|
|8589953347|      0.2|       0.7|Other Region|
|     53043|      3.0|       0.7|Other Region|
|8589937053|      1.0|       0.7|Other Region|
|     53495|      1.0|       0.7|Other Region|
|8589950587|      0.4|       0.7|Other Region|
|     52689|      1.0|       0.7|Other Region|
|8589960927|      1.0|       0.7|Other Region|
|8589936293|      2.0|       0.7|Other Region|
|8589947718|      4.0|       0.7|Other Region|
|8589958969|      2.0|       0.7|Other Region|
|8589966880|      2.0|       0.7|Other Region|
|8589960554|      1.0|       0.7|Other Region|
|8589967306|      1.0|       0.7|Other Region|
|8589938569| 

In [56]:
df1 = df1.withColumn("id",df1["var_id"].cast(LongType()))

In [57]:
df1.printSchema()

root
 |-- var_id: string (nullable = true)
 |-- var_value: double (nullable = true)
 |-- Efficiency: double (nullable = true)
 |-- Model: string (nullable = true)
 |-- id: long (nullable = true)



In [58]:
df2 = Optimization_T_Final_temp.select([col for col in Optimization_T_Final_temp.columns if col not in {"Predicted_new","EPB_new","Objective_Fun","Lower_Limit","var_name","part_id"}]).join(df1,"id","inner")

In [59]:
def traffic_allow(x,y):
    return (x/y)

traffic_allow_udf = udf(traffic_allow,DoubleType())

df2 = df2.withColumn("traffic_allow",traffic_allow_udf(df2.var_value,df2.Average_Weekly_Traffic))

In [62]:
df2.printSchema()

root
 |-- Hotel_ID: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Region_Bucket: string (nullable = true)
 |-- Market_Bucket: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Model_Name: string (nullable = true)
 |-- City_Bucket: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Hotel_Type: string (nullable = true)
 |-- Hotel_Type_Bucket: string (nullable = true)
 |-- Star_Rating: string (nullable = true)
 |-- Star_Rating_Bucket: string (nullable = true)
 |-- LOS_Bucket: string (nullable = true)
 |-- BW_Bucket: string (nullable = true)
 |-- seller_rank: integer (nullable = true)
 |-- Seller_Rank_Bucket: string (nullable = true)
 |-- Traffic: integer (nullable = true)
 |-- Cost: double (nullable = true)
 |-- CPC: double (nullable = true)
 |-- Average_Weekly_Traffic: double (nullable = true)
 |-- Booking: integer (nullable = true)
 |-- Gross_Profit: double (nullable = true)
 |-- EPB: double (nullable = true)
 |-- Predicted: double

In [67]:
df2 = df2.select([col for col in df2.columns if col not in {"id","var_id","Model"}])
df2 = df2.withColumnRenamed("var_value","Lp_solution")

In [68]:
df2.printSchema()

root
 |-- Hotel_ID: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Region_Bucket: string (nullable = true)
 |-- Market_Bucket: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Model_Name: string (nullable = true)
 |-- City_Bucket: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Hotel_Type: string (nullable = true)
 |-- Hotel_Type_Bucket: string (nullable = true)
 |-- Star_Rating: string (nullable = true)
 |-- Star_Rating_Bucket: string (nullable = true)
 |-- LOS_Bucket: string (nullable = true)
 |-- BW_Bucket: string (nullable = true)
 |-- seller_rank: integer (nullable = true)
 |-- Seller_Rank_Bucket: string (nullable = true)
 |-- Traffic: integer (nullable = true)
 |-- Cost: double (nullable = true)
 |-- CPC: double (nullable = true)
 |-- Average_Weekly_Traffic: double (nullable = true)
 |-- Booking: integer (nullable = true)
 |-- Gross_Profit: double (nullable = true)
 |-- EPB: double (nullable = true)
 |-- Predicted: double

In [20]:
Opt_R = sqlContext.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true') \
.option('mode', 'DROPMALFORMED').load('/home/affine/Expedia/Data/Opt.csv')


In [21]:
Opt_R.printSchema()

root
 |-- Hotel_ID: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Region_Bucket: string (nullable = true)
 |-- Market_Bucket: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Model_Name: string (nullable = true)
 |-- City_Bucket: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Hotel_Type: string (nullable = true)
 |-- Hotel_Type_Bucket: string (nullable = true)
 |-- Star_Rating: string (nullable = true)
 |-- Star_Rating_Bucket: string (nullable = true)
 |-- LOS_Bucket: string (nullable = true)
 |-- BW_Bucket: string (nullable = true)
 |-- seller_rank: integer (nullable = true)
 |-- Seller_Rank_Bucket: string (nullable = true)
 |-- Traffic: integer (nullable = true)
 |-- Cost: double (nullable = true)
 |-- CPC: double (nullable = true)
 |-- Average_Weekly_Traffic: double (nullable = true)
 |-- Booking: integer (nullable = true)
 |-- Gross_Profit: double (nullable = true)
 |-- EPB: double (nullable = true)
 |-- Predicted: double

In [27]:
Opt_R_1 = Opt_R.select("Hotel_ID","LOS_Bucket","BW_Bucket","Lp_solution")
df2_1 = df2.select("Hotel_ID","LOS_Bucket","BW_Bucket","var_value")

In [28]:
comp_df = Opt_R_1.join(df2_1,["Hotel_ID","LOS_Bucket","BW_Bucket"],"inner")

In [29]:
comp_df.show(100)

+--------+----------+---------+-----------+---------+
|Hotel_ID|LOS_Bucket|BW_Bucket|Lp_solution|var_value|
+--------+----------+---------+-----------+---------+
|      71|         1|        1|        0.2|      0.2|
|     345|         1|      5-7|        3.0|      3.0|
|    1112|         5|   91-120|        0.5|      0.5|
|    1624|         2|    31-40|        0.4|      0.4|
|    4590|         1|    41-60|        1.0|      1.0|
|    4686|         4|    21-30|        1.0|      0.5|
|    6112|         5|   91-120|        0.5|      0.5|
|    6226|         4|      5-7|        0.4|      0.4|
|    6644|         3|    41-60|        0.4|      0.4|
|    7797|         1|    31-40|        0.2|      0.2|
|    8478|         4|    31-40|        0.3|      0.3|
|    9017|         2|    41-60|        0.2|      0.2|
|    9228|         3|     >240|        1.0|      1.0|
|    9572|         2|      5-7|        0.2|      0.2|
|   10605|         1|    21-30|        0.3|      0.3|
|   10859|         3|    21-